In [1]:
clean_up=True # removes gams-related files in work-folder if true
%run StdPackages.ipynb
name = 'V01'
directory['gams'] = directory['gams']+'\\'+name
OS = DB2Gams.OrdSet

# Partial equilibrium model with technologies, v 0.1 [(link to paper)](https://www.overleaf.com/read/stkxjdbsyzjq)

*Load data from ```V1_Data.ipynb```*

In [2]:
db = DataBase.GPM_database(pickle_path = directory['data']+f"\\{name}_DB")

## 2: Models

We define four potential model states, each constitutes a square system. All groups that are not included in "endogenous" groups are per default exogenous; thus, we are mostly interested in specifying the right endogenous groups and models.
* "B": The model is solved as if already calibrated.
    * Endogenous groups: ```G_{name}_NT_endo_always```, ```G_{name}_NT_endo_base```, ```G_{name}_T_endo_always```, ```G_{name}_T_endo_base```, ```G_{name}_ACC_endo_base```.
    * Exogenous groups : ```G_{name}_NT_exo_always``` , ```G_{name}_NT_exo_base``` , ```G_{name}_T_exo_always```, ```G_{name}_T_exo_base```.
    * Models blocks: ```M_{name}_NT```, ```M_{name}_T_always```,```M_{name}_T_base```, ```M_{name}_ACC```.
* "CalibTech": The model calibrates the technology part.
    * Endogenous groups: ```G_{name}_T_endo_always```, ```G_{name}_T_exo_base```.
    * Exogenous groups : ```G_{name}_T_exo_always```, ```G_{name}_T_endo_base```, ```G_{name}_NT_exo_always```.
    * Model blocks: ```M_{name}_T_always```. (Note that the block ```M_{name}_T_base``` is implied if calibration data is consistent)
* "CalibGivenT": The model assumes technology part is calibrated, and calibrates the rest of the model.
    * Endogenous groups: ```G_{name}_NT_endo_always```, ```G_{name}_NT_exo_base```
    * Exogenous groups: ```G_{name}_NT_exo_always```, ```G_{name}_NT_endo_base```, ```G_{name}_ACC_endo_base```, ```G_{name}_T_exo_always```, ```G_{name}_T_exo_base```, ```G_{name}_T_endo_base```.
    * Model blocks: ```M_{name}_NT```, ```M_{name}_ACC```.
* "CalibAll": The model calibrates both parts simultanoeusly:
    * Endogenous groups: ```G_{name}_NT_endo_always```, ```G_{name}_NT_exo_base```, ```G_{name}_T_endo_always```, ```G_{name}_T_exo_base```.
    * Exogenous groups: Residual.
    * Model blocks: ```M_{name}_NT```, ```M_{name}_T_always```, ```M_{name}_ACC```.

### 2.1. Settings

*All files, functions, groups, blocks:*

In [3]:
files = {f"{name}_{k}.gms": directory['gams'] for k in ('groups','blocks')}
functions = {f"{name}_functions.gms": directory['gams']}
groups = OS(regex_gms.groups_from_file(directory['gams']+f"\\{name}_groups.gms"))
blocks = OS(regex_gms.blocks_from_file(directory['gams']+f"\\{name}_blocks.gms"))

*Model settings: Set endogenous groups, blocks, and exogenous groups for all four*

In [4]:
B = {'name': "B",
     'g_endo': OS([f"G_{name}_NT_endo_always", f"G_{name}_NT_endo_base", f"G_{name}_T_endo_always", f"G_{name}_T_endo_base", f"G_{name}_ACC_endo_base"]),
     'blocks': blocks}
B['g_exo'] = groups-B['g_endo']

CT = {'name': f"CalibTech",
      'g_endo': OS([f"G_{name}_T_endo_always", f"G_{name}_T_exo_base"]),
      'blocks': OS([f"M_{name}_T_always"])}
CT['g_exo'] = groups-CT['g_endo']

CgT = {'name':f"CalibGivenT",
       'g_endo': OS([f"G_{name}_NT_endo_always", f"G_{name}_NT_exo_base"]),
       'blocks': blocks-CT['blocks']-OS([f"M_{name}_T_base"])}
CgT['g_exo'] = groups-CgT['g_endo']

CA = {'name': f"CalibAll",
      'g_endo' : OS([f"G_{name}_NT_endo_always", f"G_{name}_NT_exo_base", f"G_{name}_T_endo_always", f"G_{name}_T_exo_base"]),
      'blocks': blocks-OS([f"M_{name}_T_base"])}
CA['g_exo'] = groups-CA['g_endo']

*Set up ```gams_settings``` with the four states*

In [5]:
gs = DB2Gams.gams_settings(work_folder = directory['work'], **{'name':name, 'data_folder': directory['gams']})
for s in (B,CT,CgT,CA):
    gs.conf[s['name']] = gs.std_configuration(state=s['name'])
    [gs.conf[s['name']].__setitem__(k, s[k]) for k in ('g_endo','g_exo','blocks')];
gs.functions = functions
gs.mfiles = files

*Set up ```gams_model_py```:*

In [6]:
gm_py = DB2Gams.gams_model_py(gsettings=gs)

*Add data:*

In [7]:
gs.add_database(db) # add data
gm_py.database.merge_internal() # write to a gdx file in the working directory

*apply standard settings:*

In [8]:
apply_settings(gm_py)

### 2.2. Run/execute model

Set up model from settings:

In [9]:
gm = DB2Gams.gams_model(gsettings = gs)

We can run the model in various ways now, by setting the relevant state. Can set to {```"B"```, ```"TroubleShoot_B"```, ```"CalibTech"```. ```"CT_CgT"```, ```"TroubleShoot_CT_CgT"```}. If we change ```robust = True``` calibration methods will use a ```sneaky_solve``` method that takes a bit longer, but is more likely to work. 

In [22]:
runlike = "CT_CgT"
robust = False

Different run modes:

#### 2.2.1. Baseline

Solve the full model in baseline mode:

In [11]:
if runlike == "B":
    # gm.setstate = 'B' # default setting is 'B'.
    gm.run()

#### 2.2.2. Baseline, troubleshoot

If the baseline model is not square, solve a trivial NLP problem instead:

In [12]:
if runlike == "TroubleShoot_B":
    gs.set_conf('solve',f"""@SolveEmptyNLP({gs.get_conf('name')})""") # use to troubleshoot non-square system; solve model as NLP.
    gm.run()

#### 2.2.3. Calibrate technology part

In [13]:
if runlike == "CalibTech":
    gs.setstate = "CalibTech"
    gm.run()

Inspect potential differences (if the technology type is "logit", the ```V1_Data.ipynb``` should have already calibrated the model):

In [14]:
if runlike == "CalibTech":
    var = 'mu'
    diff = (gm.out_db.get(var)-db.get(var)).astype(float).round(decimals = 7)
    diff_neq0 = diff[diff!=0]
    if diff_neq0.empty:
        print(f"No changes were made to {var} in calibration")
    else:
        pd.DataFrame({'Calibrated': gm.out_db.get(var)[gm.out_db.get(var).index.isin(diff_neq0.index)], 'Before': db.get(var)[db.get(var).index.isin(diff_neq0.index)]}).plot.bar(figsize=(8,6));

#### 2.2.4. Calibrate technology part, then calibrate the rest:

Get the relevant text used to define models/groups to run in state "CalibGivenT":

In [15]:
if runlike == "CT_CgT":
    gs.setstate = "CalibGivenT"
    text_CgT = DB2Gams.run_text(**{k:gs.get_conf(k) for k in ('g_exo','g_endo','blocks','name','solve')}) # store text used to change model state and re-run solve statement

Set the state to "CalibTech" and solve first in this state, and add the text ```text_CgT``` to the end of the program:

In [16]:
if runlike == "CT_CgT":
    gs.setstate = "CalibTech"
    gm.run(kwargs_write={'end': text_CgT}) # kwargs_write makes ad-hoc adjustments to the automated run_file produced by gm. The 'end' keyword states that the adjustment is made in the end.

#### 2.2.5. Troubleshoot version:

Get the relevant text used to define models/groups to run in state "CalibGivenT"; add that instead of solving in this state, define a trivial NLP block:

In [17]:
if runlike == "TroubleShoot_CT_CgT":
    gs.setstate = "CalibGivenT"
    gs.set_conf('solve',f"""@SolveEmptyNLP({gs.get_conf('name')})""") # use to troubleshoot non-square system; solve model as NLP.
    text_CgT = DB2Gams.run_text(**{k:gs.get_conf(k) for k in ('g_exo','g_endo','blocks','name','solve')}) # store text used to change model state and re-run solve statement

Set the state to "CalibTech" and solve first in this state, and add the text ```text_CgT``` to the end of the program:

In [18]:
if runlike == "TroubleShoot_CT_CgT":
    gs.setstate = "CalibTech"
    gm.run(kwargs_write={'end': text_CgT}) # kwargs_write makes ad-hoc adjustments to the automated run_file produced by gm. The 'end' keyword states that the adjustment is made in the end.